### LIMPIEZA Y TRANSFORMACIÓN DE DATOS

In [66]:
# Cargar librerías
import pandas as pd  # Manejo de dataframes

In [67]:
# Cargar data
df_emp = pd.read_csv("data/employee_survey_data.csv")
df_gen = pd.read_csv("data/general_data.csv", sep=";")
df_in_time = pd.read_csv("data/in_time.csv")
df_manager = pd.read_csv("data/manager_survey_data.csv")
df_out_time = pd.read_csv("data/out_time.csv")
df_retirement = pd.read_csv("data/retirement_info.csv", sep=";")

In [68]:
# Renombrar columnas
df_in_time = df_in_time.rename(columns={"Unnamed: 0": "EmployeeID"})
df_out_time = df_out_time.rename(columns={"Unnamed: 0": "EmployeeID"})

# Establecer índices
df_emp = df_emp.set_index("EmployeeID")
df_gen = df_gen.set_index("EmployeeID")
df_in_time = df_in_time.set_index("EmployeeID")
df_manager = df_manager.set_index("EmployeeID")
df_out_time = df_out_time.set_index("EmployeeID")
df_retirement = df_retirement.set_index("EmployeeID")

# Convertir a formato fecha
df_in_time = df_in_time.apply(pd.to_datetime)
df_out_time = df_out_time.apply(pd.to_datetime)

df_retirement["retirementDate"] = pd.to_datetime(
    df_retirement["retirementDate"], format="%d/%m/%Y"
)

# Eliminar columnas innecesarias ya que son redundantes
df_gen = df_gen.drop(["Over18", "StandardHours", "EmployeeCount"], axis=1)

In [69]:
# Estos nulos se tratan antes ya que las bases tienen diferentes dimensiones, por lo que
# al unir estos nulos se juntarían con los nulos que no se han retirado


# Los nulos de la columna resignationReason se reemplazan por "Fired" porque todos los nulos son los que echaron
df_retirement["resignationReason"] = df_retirement["resignationReason"].fillna("Fired")

In [70]:
# Unir todos los dataframes con el índice con merge
df = (
    df_gen.merge(df_emp, on="EmployeeID", how="outer")
    .merge(df_manager, on="EmployeeID", how="outer")
    .merge(df_retirement, on="EmployeeID", how="outer")
)


### Renombrar Variable objetivo
df = df.rename(columns={"Attrition": "target"})

### Comprobación de unión exitosa
print("Comprobación de unión exitosa")
print(df.shape)
print("------------------------------")

## Ver los nulos en el dataframe
df.isnull().sum()

Comprobación de unión exitosa
(4410, 28)
------------------------------


Age                           0
BusinessTravel                0
Department                    0
DistanceFromHome              0
Education                     0
EducationField                0
Gender                        0
JobLevel                      0
JobRole                       0
MaritalStatus                 0
MonthlyIncome                 0
NumCompaniesWorked           19
PercentSalaryHike             0
StockOptionLevel              0
TotalWorkingYears             9
TrainingTimesLastYear         0
YearsAtCompany                0
YearsSinceLastPromotion       0
YearsWithCurrManager          0
EnvironmentSatisfaction      25
JobSatisfaction              20
WorkLifeBalance              38
JobInvolvement                0
PerformanceRating             0
target                     3699
retirementDate             3699
retirementType             3699
resignationReason          3699
dtype: int64

In [73]:
#### TRATAMIENTO DE NULOS

# Se eliminan los registros nulos de NumCompaniesWorked y TotalWorkingYears porque son pocos y no se ven patrones
df = df.dropna(subset=["NumCompaniesWorked", "TotalWorkingYears"])
# Los nan de la columna "Attrition" se reemplazan por "No"
df["target"] = df["target"].fillna("No")


# NULOS POR UNIR DATASETS: Los nulos que se ven es porque se unieron con una base de datos de retirados
# entonces los nulos que hay es porque hay empleados que no se han retirado

# Los nan de retirementType se reemplazan por "No"
df["retirementType"] = df["retirementType"].fillna("No")
# Los nan de resignationReason se reemplazan por "No"
df["resignationReason"] = df["resignationReason"].fillna("No")
# Los nan de retirementDate se dejan como están para facilitar la manipulación de datos


# Los nulos de EnvironmentSatisfaction, JobSatisfaction, WorkLifeBalance se imputaran más adelante


## Ver los nulos en el dataframe
df.isnull().sum()

Age                           0
BusinessTravel                0
Department                    0
DistanceFromHome              0
Education                     0
EducationField                0
Gender                        0
JobLevel                      0
JobRole                       0
MaritalStatus                 0
MonthlyIncome                 0
NumCompaniesWorked            0
PercentSalaryHike             0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
YearsAtCompany                0
YearsSinceLastPromotion       0
YearsWithCurrManager          0
EnvironmentSatisfaction      25
JobSatisfaction              20
WorkLifeBalance              37
JobInvolvement                0
PerformanceRating             0
target                        0
retirementDate             3677
retirementType                0
resignationReason             0
dtype: int64

In [75]:
# Transformar variable objetivo a binaria
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
df["target"] = enc.fit_transform(df["target"])

In [76]:
# Guardar dataframe en csv para usarlo en el siguiente notebook
df.to_csv("data/df.csv", index=True)